In [64]:
import pandas as pd
import holidays
from holidays.constants import PUBLIC, UNOFFICIAL
from pathlib import Path

In [ ]:
start_ts = pd.Timestamp("1990-01-01")
end_ts = pd.Timestamp("2024-12-31")
years = range(start_ts.year, end_ts.year + 1)

out_dir = Path("..") / "data" / "holidays" / "raw"
out_dir.mkdir(parents=True, exist_ok=True)

In [66]:
us_public_holidays = holidays.country_holidays("US", categories=PUBLIC, years=years)
us_unofficial_holidays = holidays.country_holidays("US", categories=UNOFFICIAL, years=years)

nyse_holidays = holidays.financial_holidays("NYSE", years=years)

In [72]:
all_dates = sorted(
    set(us_public_holidays.keys())
    | set(us_unofficial_holidays.keys())
    | set(nyse_holidays.keys())
)
df = pd.DataFrame({"timestamp": pd.to_datetime(all_dates)})
df = df[(df["timestamp"] >= start_ts) & (df["timestamp"] <= end_ts)].reset_index(drop=True)

date_keys = df["timestamp"].map(lambda ts: ts.date())
df["publicHolidaysUS"] = date_keys.map(us_public_holidays.get)
df["unofficialHolidaysUS"] = date_keys.map(us_unofficial_holidays.get)
df["financialHolidaysNYSE"] = date_keys.map(nyse_holidays.get)

In [73]:
df

,timestamp,publicHolidaysUS,unofficialHolidaysUS,financialHolidaysNYSE
0,1990-01-01,New Year's Day,None,New Year's Day
1,1990-01-15,Martin Luther King Jr. Day,None,None
2,1990-02-02,None,Groundhog Day,None
3,1990-02-14,None,Valentine's Day,None
4,1990-02-19,Washington's Birthday,None,Washington's Birthday
...,...,...,...,...
654,2024-10-31,None,Halloween,None
655,2024-11-05,None,Election Day,None
656,2024-11-11,Veterans Day,None,None
657,2024-11-28,Thanksgiving Day,None,Thanksgiving Day


In [ ]:
us_df   = df[["timestamp", "publicHolidaysUS", "unofficialHolidaysUS"]]
nyse_df = df[["timestamp", "financialHolidaysNYSE"]]

In [74]:
us_df.to_csv(out_dir / "us_holidays.csv", index=False)
us_df.to_parquet(out_dir / "us_holidays.parquet", index=False)

nyse_df.to_csv(out_dir / "nyse_holidays.csv", index=False)
nyse_df.to_parquet(out_dir / "nyse_holidays.parquet", index=False)